In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.optimizers import SGD
import numpy as np
import pathlib
import matplotlib.pyplot as plt
from os.path import dirname, join as pjoin
import scipy.io as sio
import h5py
import hdf5storage
import random
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import load_model
from PIL import Image
from matplotlib.image import imread
import os
from tensorflow.keras import applications
from sklearn.datasets import load_svmlight_file
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import RegexpTokenizer

In [2]:
DATASET_URL = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

In [3]:
data_root_orig = tf.keras.utils.get_file(origin=DATASET_URL, fname='aclImdb', untar=True)
data_root = pathlib.Path(data_root_orig)
print(data_root)

nltk.download('punkt')

/home/maxim/.keras/datasets/aclImdb


[nltk_data] Downloading package punkt to /home/maxim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
PATH_TO_TRAIN = str(data_root) + '/train'
PATH_TO_TEST =str(data_root) + '/test'

In [5]:
import re

NUM_TOKEN = '<num>'
UNKNOWN_TOKEN = '<unk>'

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', raw_html)
    return cleantext

def review_to_indices(text, vocab):
    indecies = []
    for text_word in text:
        word = text_word.lower()
        index = -1
        # try find word in vocab
        try:
            index = vocab.index(word)
        except:
            # if we can't find it - try to parse it to int
            try:
                i = int(word)
                index = vocab.index(NUM_TOKEN)
            except:
                # else it unknown
                index = vocab.index(UNKNOWN_TOKEN)
        indecies.append(index)
    return indecies
                
   

In [21]:
# load data

# load vocab

vocab = []

with open(str(data_root) + '/imdb.vocab', 'r') as file:
    vocab = file.read().splitlines()
    vocab.append(NUM_TOKEN)
    vocab.append(UNKNOWN_TOKEN)
    print('Added <NUM> and <UNK> tokens')
    print('Vocab len =', len(vocab))
    
tokenizer = RegexpTokenizer(r'\w+')

total_review_len = 0
total_review_count = 0

reviews = []

for review_name in os.listdir(PATH_TO_TRAIN + '/pos'):
    mark = int(review_name.split('_')[1].split('.')[0])
    path = PATH_TO_TRAIN + '/pos/' + review_name
    with open(path, 'r') as file:
        data = file.read()
        clean_review = cleanhtml(data)
        indecies = review_to_indices(tokenizer.tokenize(clean_review), vocab)
        total_review_len += len(indecies)
        total_review_count += 1
        reviews.append((indecies, mark))

print(len(reviews))
print(reviews[10])

Added <NUM> and <UNK> tokens
Vocab len = 89529
12500
([9, 5, 2, 49, 214, 70, 621, 1527, 15, 82, 6, 5, 2, 931, 1127, 0, 356, 1182, 22, 321, 1, 0, 109, 5, 255, 6, 39, 19, 178, 48, 252, 6515, 9118, 6, 528, 79, 7, 5495, 59, 1469, 6, 388, 2, 526, 3, 2, 145, 1425, 418, 1, 19, 2, 1527, 15, 16, 3, 254, 6, 5, 2, 1527, 1, 2, 62, 48, 26, 14, 314, 1281, 461, 809, 1, 1357, 1, 30, 0, 93, 0, 381, 5, 902, 103, 44, 13, 9, 26], 8)


In [ ]:
for review_name in os.listdir(PATH_TO_TRAIN + '/pos'):
    with open(path, 'r') as file:
        data = file.read()
        clean_review = cleanhtml(data)

In [7]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

y_train

Loading data...


Using TensorFlow backend.


array([1, 0, 0, ..., 0, 1, 0])

In [19]:
'0_10.txt'

'10'